In [1]:
import os
import datetime
import math
import pandas as pd
from collections import Counter
from sym_creator import Symbol_Creator

In [2]:
out_dir = 'Created'
symbol_name = 'test_symbol'

# Create symbol from CSV

In [5]:
csv_file = 'simple_import3.csv'
csv = pd.read_csv(csv_file, delimiter=';')

In [6]:
# df = csv.filter(['Pin Label', 'Pin Number', 'Pin Type'])
# df.to_csv('simple_import.csv', index=False, sep=';')

In [7]:
# # TODO: SPI, I2C, etc.
# def get_bus_signals(df):
#     pass

In [8]:
creator = Symbol_Creator(out_dir=out_dir, out_symbol_name='export3')
creator.symbol_from_csv(csv_file, symbol_name)
creator.export_symbol()

900x2700.0


In [9]:
w = creator.estimate_box_width(list(csv['Pin Label'].values))
test = creator.get_diff_pairs(csv)
test = creator.predict_inverted_to_df(test)
test = creator.predict_side_to_df(test, creator.default_cond_list)
test = creator.get_coordinates(test, w)
test

,Pin Label,Pin Number,Pin Type,Diff,Inverted,Side,group,NC,PAD,GND,PWR,INV,sort,Type_Change,gap,x,y
0,EPAD,21,GROUND,-1,False,2,-1,1,0,0,0,1,0.0,False,0.0,0,0.0
1,GND1,8,GROUND,-1,False,2,0,1,1,0,0,1,1.0,False,1.0,0,200.0
2,GND2,18,GROUND,-1,False,2,0,1,1,0,0,1,2.0,False,1.0,0,400.0
3,SNS,2,IN,-1,False,2,-1,1,1,1,0,1,3.0,True,1.0,0,600.0
4,FB,3,IN,-1,False,2,-1,1,1,1,0,1,4.0,False,0.0,0,700.0
5,50mV,5,ANALOG,-1,False,2,-1,1,1,1,0,1,5.0,True,1.0,0,900.0
6,100mV,6,ANALOG,-1,False,2,-1,1,1,1,0,1,6.0,False,0.0,0,1000.0
7,200mV,7,ANALOG,-1,False,2,-1,1,1,1,0,1,7.0,False,0.0,0,1100.0
8,400mV,9,ANALOG,-1,False,2,-1,1,1,1,0,1,8.0,False,0.0,0,1200.0
9,800mV,10,ANALOG,-1,False,2,-1,1,1,1,0,1,9.0,False,0.0,0,1300.0


In [10]:
# import re
# p = re.compile('[a-zA-Z]{2,}[0-9]{1,}')
# matches = []
# for v in csv['Pin Label'].values:
#     m = p.match(v)
#     if m != None:
#         match_val = m.group()
#         matches += [''.join([v for v in match_val if v.isalpha()])]
# groups = [k for k, v in Counter(matches).items() if v > 1]

In [11]:
# [[i for i, g in enumerate(groups) if v.startswith(g)][0] if v.startswith(tuple(groups)) else -1 for v in csv.loc[:, 'Pin Label'].values]

In [12]:
# def simplify_csv(csv_file, out_name='simple_import.csv'):
#     csv = pd.read_csv(csv_file, delimiter=';')
#     csv = csv.filter(['Pin Label', 'Pin Number', 'Pin Type'])
#     csv.to_csv(out_name, index=False, sep=';')

In [13]:
# simplify_csv('import2.csv', out_name='simple_import3.csv')